In [1]:
A, B, C = "K", "L", "Z"
sA, sB, sC = "RR", "LR", "RM"

In [2]:
p = MixedIntegerLinearProgram(maximization = True)
x = p.new_variable(binary = True)

In [3]:
p.set_objective(x["RM", "K"] * 2000 + x["RM", "L"] * 2200 + x["RM", "Z"] * 750 + x["RM", "O"] * 1800 +
                x["LR", "K"] * 10000 + x["LR", "L"] * 5500 + x["LR", "Z"] * 750 + x["LR", "O"] * 1800 +
                x["RR", "K"] * 5000 + x["RR", "L"] * 5500 + x["RR", "Z"] * 750 + x["RR", "O"] * 1800)

In [4]:
sorte = ["RM", "LR", "RR"]
stranke = ["K", "L", "Z", "O"]

In [5]:
for i in sorte:
    p.add_constraint(sum(x[i, j] for j in stranke) <= 1)
for j in stranke:
    p.add_constraint(sum(x[i, j] for i in sorte) <= 1)
p.add_constraint(x["RM", "O"] + x["LR", "O"] == 0)
p.add_constraint(x["LR", "K"] <= x["RM", "L"])
p.add_constraint(sum(x[i, "O"] + x[i, "Z"] - x[i, "K"] for i in sorte) >= 0)
p.add_constraint(x[sA, A] + x[sB, B] + sum(x[sC, j] for j in stranke if j != C) <= 2)

In [6]:
p.solve()

14000.0

In [7]:
res = p.get_values(x)
res

{('LR', 'K'): 1.0,
 ('LR', 'L'): 0.0,
 ('LR', 'O'): 0.0,
 ('LR', 'Z'): 0.0,
 ('RM', 'K'): 0.0,
 ('RM', 'L'): 1.0,
 ('RM', 'O'): 0.0,
 ('RM', 'Z'): 0.0,
 ('RR', 'K'): 0.0,
 ('RR', 'L'): 0.0,
 ('RR', 'O'): 1.0,
 ('RR', 'Z'): 0.0}

In [8]:
{i: j for (i, j), v in res.items() if v == 1}

{'LR': 'K', 'RM': 'L', 'RR': 'O'}